In [147]:
import subprocess
import glob
import os
from os.path import exists
import re
import pandas as pd
from multiprocessing import Pool
import pyreadr

In [148]:
WGCNA_path = './WGCNA_ATAC_integration'

In [149]:
non_variable_peaks_in_RNA_seq = pyreadr.read_r(os.path.join(WGCNA_path, "non_variable_peaks_in_RNA_seq.rds"))
non_variable_peaks_in_RNA_seq = non_variable_peaks_in_RNA_seq[None]
len(non_variable_peaks_in_RNA_seq)

24266

In [150]:
# Load the RDS file into Python
result = pyreadr.read_r(os.path.join(WGCNA_path, "connectivity_and_peak_info_DM1_DM2d.rds"))
df = result[None]


In [151]:
# Create an empty dictionary to store the subsets
module_dict = {}

# Loop through the unique values in the 'module' column
for module in df['module'].unique():
    # Subset the dataframe where 'module' equals the current unique value
    module_dict[module] = df[df['module'] == module]

# Now 'module_dict' contains each module as a key with its corresponding subset dataframe as its value.


In [152]:
module_dict['MEcyan']


,gene,connectivity_rowSums_adj,module,gene_name,DM6/DM7,DM1/DM2d,DM10/DM11,AvrRpt2-6h,AvrRpt2-12h,AvrRpt2-20h,...,Fold,p.value,FDR,annotation,geneChr,geneStart,geneEnd,geneLength,geneStrand,distanceToTSS
0,AT1G63720,1954.528305,MEcyan,AT1G63720,1.935779,3.970436,6.033027,0.887911,1.510569,3.775730,...,-0.406409,2.593965e-07,1.476102e-06,Promoter (<=1kb),1,23635745,23637577,1833,2,699.0
1,AT3G54150,1952.887628,MEcyan,EFD,3.815566,5.122284,8.439073,1.157111,3.410757,5.334764,...,0.520665,5.630124e-08,3.492766e-07,Promoter (<=1kb),3,20050564,20052931,2368,2,0.0
2,AT5G48657,1947.970337,MEcyan,NOI10,3.501474,4.454391,8.090660,1.011472,2.203670,3.545495,...,0.270921,1.657224e-04,6.039474e-04,Promoter (<=1kb),5,19733882,19735726,1845,2,0.0
3,AT5G48657,1947.970337,MEcyan,NOI10,3.501474,4.454391,8.090660,1.011472,2.203670,3.545495,...,0.222130,2.069287e-03,5.910763e-03,Promoter (<=1kb),5,19733882,19735726,1845,2,838.0
4,AT4G23700,1947.947682,MEcyan,CHX17,6.202520,6.294546,9.241291,0.900020,3.321431,6.899931,...,1.000460,2.808698e-32,8.609768e-31,Promoter (<=1kb),4,12342376,12346051,3676,2,-708.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6067,AT5G61790,664.442523,MEcyan,CNX1,1.128610,1.005367,1.013328,0.093627,1.096341,0.998485,...,-0.510068,3.789737e-12,3.583663e-11,Promoter (<=1kb),5,24827063,24829900,2838,2,0.0
6073,AT4G24190,603.862556,MEcyan,SHD,1.396126,0.295383,0.784250,-0.194620,0.850157,0.795426,...,-0.625392,3.304481e-30,9.103409e-29,Promoter (<=1kb),4,12551552,12556058,4507,2,0.0
6074,AT3G41768,571.912538,MEcyan,AT3G41768,-0.812099,1.071715,-0.107859,1.196581,1.036284,-0.368824,...,-0.598999,1.260891e-06,6.518600e-06,Promoter (<=1kb),3,14197677,14199484,1808,1,443.0
6075,AT2G16660,513.021525,MEcyan,AT2G16660,2.005898,-0.166410,-0.635497,-0.099147,0.005229,0.512974,...,-0.474237,4.214350e-10,3.318071e-09,Promoter (1-2kb),2,7218654,7221890,3237,2,-1727.0


In [153]:
module_dict['MEcyan']['module'].unique().tolist()[0]


'MEcyan'

In [154]:
#first 2 rows
module_dict['MEcyan'].iloc[0:2]

,gene,connectivity_rowSums_adj,module,gene_name,DM6/DM7,DM1/DM2d,DM10/DM11,AvrRpt2-6h,AvrRpt2-12h,AvrRpt2-20h,...,Fold,p.value,FDR,annotation,geneChr,geneStart,geneEnd,geneLength,geneStrand,distanceToTSS
0,AT1G63720,1954.528305,MEcyan,AT1G63720,1.935779,3.970436,6.033027,0.887911,1.510569,3.775730,...,-0.406409,2.593965e-07,1.476102e-06,Promoter (<=1kb),1,23635745,23637577,1833,2,699.0
1,AT3G54150,1952.887628,MEcyan,EFD,3.815566,5.122284,8.439073,1.157111,3.410757,5.334764,...,0.520665,5.630124e-08,3.492766e-07,Promoter (<=1kb),3,20050564,20052931,2368,2,0.0


In [155]:
#get 
module_dict['MEcyan']['gene']

0       AT1G63720
1       AT3G54150
2       AT5G48657
3       AT5G48657
4       AT4G23700
          ...    
6067    AT5G61790
6073    AT4G24190
6074    AT3G41768
6075    AT2G16660
6076    AT2G16660
Name: gene, Length: 2859, dtype: object

In [45]:
WGCNA_path

'./WGCNA_ATAC_integration'

In [156]:
import os
import pandas as pd

def process_all_csv_to_single_bed_dir(module_dict, df, WGCNA_path):
    # Define the bed files directory
    bed_files_dir = os.path.join(WGCNA_path, 'bed_files')
    
    # Ensure the bed files directory exists
    os.makedirs(bed_files_dir, exist_ok=True)

    # Iterate over all keys in the module_dict
    for module_name in module_dict.keys():
        # Filter the DataFrame for the current module and FDR < 0.05
        module_df = df[(df['module'] == module_name) & (df['FDR'] < 0.05)]

        # Proceed only if there are entries to write
        if not module_df.empty:
            # Construct the path for the BED file within the 'bed_files' directory
            bed_file = os.path.join(bed_files_dir, f"{module_name}.bed")

            # Select the required columns and save to BED format
            module_df[['seqnames', 'start', 'end', 'width', 'strand']].to_csv(
                bed_file, sep='\t', header=False, index=False)

            print(f"Generated BED file at: {bed_file}")


process_all_csv_to_single_bed_dir(module_dict, df, WGCNA_path)


Generated BED file at: ./WGCNA_ATAC_integration/bed_files/MEcyan.bed
Generated BED file at: ./WGCNA_ATAC_integration/bed_files/MEblue.bed
Generated BED file at: ./WGCNA_ATAC_integration/bed_files/MEpurple.bed
Generated BED file at: ./WGCNA_ATAC_integration/bed_files/MEdarkred.bed
Generated BED file at: ./WGCNA_ATAC_integration/bed_files/MEgreenyellow.bed
Generated BED file at: ./WGCNA_ATAC_integration/bed_files/MEtan.bed
Generated BED file at: ./WGCNA_ATAC_integration/bed_files/MEdarkgreen.bed
Generated BED file at: ./WGCNA_ATAC_integration/bed_files/MEsalmon.bed
Generated BED file at: ./WGCNA_ATAC_integration/bed_files/MEmidnightblue.bed
Generated BED file at: ./WGCNA_ATAC_integration/bed_files/MEdarkturquoise.bed


In [157]:
import os
import pandas as pd
from Bio import SeqIO

def process_all_csv_to_fasta_dir(module_dict, df, WGCNA_path, genome_path):
    # Define the fasta files directory
    fasta_files_dir = os.path.join(WGCNA_path, 'fasta_files')
    
    # Ensure the fasta files directory exists
    os.makedirs(fasta_files_dir, exist_ok=True)
    
    # Load the genome fasta file
    genome_sequences = SeqIO.to_dict(SeqIO.parse(genome_path, "fasta"))

    # Iterate over all keys in the module_dict
    for module_name, genes_df in module_dict.items():
        # Filter the DataFrame for the current module and FDR < 0.05
        module_df = genes_df[(genes_df['module'] == module_name) & (genes_df['FDR'] < 0.05)]

        # Proceed only if there are entries to write
        if not module_df.empty:
            # Construct the path for the fasta file within the 'fasta_files' directory
            fasta_file = os.path.join(fasta_files_dir, f"{module_name}.fasta")

            # Open the file in write mode
            with open(fasta_file, 'w') as f:
                # Iterate over rows in the module dataframe
                for index, row in module_df.iterrows():
                    # Extract sequence details
                    seq_name = row['seqnames']
                    start = int(row['start'])
                    end = int(row['end'])
                    gene_name = row['gene_name']
                    # strand = row['strand']
                    
                    # Get the corresponding sequence from the genome
                    gene_seq = genome_sequences[seq_name].seq[start-1:end]
                    
                    # # Reverse complement sequence if on negative strand
                    # if strand == '-':
                    #     gene_seq = gene_seq.reverse_complement()
                    
                    # Write to the fasta file
                    f.write(f">{gene_name}_{seq_name}:{start}-{end}\n{gene_seq}\n")

            print(f"Generated fasta file at: {fasta_file}")

# Define paths and DataFrame here
genome_path = os.path.join('/media/HDD2/Genomes/Ath_Ensembl56', 'Arabidopsis_thaliana.TAIR10.dna.toplevel.fa')

# Call the function with the necessary arguments
process_all_csv_to_fasta_dir(module_dict, df, WGCNA_path, genome_path)


Generated fasta file at: ./WGCNA_ATAC_integration/fasta_files/MEcyan.fasta
Generated fasta file at: ./WGCNA_ATAC_integration/fasta_files/MEblue.fasta
Generated fasta file at: ./WGCNA_ATAC_integration/fasta_files/MEpurple.fasta
Generated fasta file at: ./WGCNA_ATAC_integration/fasta_files/MEdarkred.fasta
Generated fasta file at: ./WGCNA_ATAC_integration/fasta_files/MEgreenyellow.fasta
Generated fasta file at: ./WGCNA_ATAC_integration/fasta_files/MEtan.fasta
Generated fasta file at: ./WGCNA_ATAC_integration/fasta_files/MEdarkgreen.fasta
Generated fasta file at: ./WGCNA_ATAC_integration/fasta_files/MEsalmon.fasta
Generated fasta file at: ./WGCNA_ATAC_integration/fasta_files/MEmidnightblue.fasta
Generated fasta file at: ./WGCNA_ATAC_integration/fasta_files/MEdarkturquoise.fasta


### for AME

In [92]:
import pandas as pd
from Bio import SeqIO

def extract_sequences_to_df(module_dict, genome_path):
    # Load the genome fasta file
    genome_sequences = SeqIO.to_dict(SeqIO.parse(genome_path, "fasta"))

    # Create a list to store sequence data
    sequences_data = []

    # Iterate over all keys in the module_dict
    for module_name, genes_df in module_dict.items():
        # Filter the DataFrame for the current module and FDR < 0.05
        module_df = genes_df[(genes_df['module'] == module_name) & (genes_df['FDR'] < 0.05)]

        # Proceed only if there are entries to add
        if not module_df.empty:
            # Iterate over rows in the module dataframe
            for index, row in module_df.iterrows():
                # Extract sequence details
                seq_name = row['seqnames']
                start = int(row['start'])
                end = int(row['end'])
                gene_name = row['gene_name']
                gene = row['gene']
                
                # Get the corresponding sequence from the genome
                gene_seq = str(genome_sequences[seq_name].seq[start-1:end])
                
                # Store the sequence data in the list
                sequences_data.append({
                    "module": module_name,
                    "seq_name": seq_name,
                    "start": start,
                    "end": end,
                    "gene" :gene,
                    "gene_name": gene_name,
                    "peak_sequence": gene_seq
                })

    # Create a DataFrame from the list of sequence data
    sequences_df = pd.DataFrame(sequences_data)

    return sequences_df

# Define paths here
genome_path = os.path.join('/media/HDD2/Genomes/Ath_Ensembl56', 'Arabidopsis_thaliana.TAIR10.dna.toplevel.fa')

# Call the function with the necessary arguments and print the resulting DataFrame
sequences_df = extract_sequences_to_df(module_dict, genome_path)
sequences_df


,module,seq_name,start,end,gene,gene_name,peak_sequence
0,MEcyan,1,23636274,23636878,AT1G63720,AT1G63720,ATTCTTGATCTGATGATGCTTCAGGGAACGTTGTTCTTAGCTTCTG...
1,MEcyan,3,20052576,20053358,AT3G54150,EFD,AGTATTATAAAGCTTACCCCAATAGCAGCCTGACCATTACCAGTTC...
2,MEcyan,5,19735475,19736700,AT5G48657,NOI10,ACTAAAACCACACCCAAAAGAAAAAAAAAAACTTACTGCCATATTT...
3,MEcyan,5,19734229,19734888,AT5G48657,NOI10,TACCTTGGGACTAGAAGGTAATGGTTGATGATGTTGGTCGATGATG...
4,MEcyan,4,12346759,12347372,AT4G23700,CHX17,AACATATAATTATATAACATTAGTATATGAGATGACATAGAAATTT...
...,...,...,...,...,...,...,...
7085,MEdarkturquoise,2,527915,529292,AT2G02100,LCR69,GATAAAAATACTCTATCCCGTGACTATTCCACTTGTTGTTACTTTT...
7086,MEdarkturquoise,2,529597,529856,AT2G02100,LCR69,AGGATTAGATTAGGATTACAACATGACTTGAGTGAAGCCACCAAAA...
7087,MEdarkturquoise,4,8020565,8021053,AT4G13850,GR-RBP2,CATCAACGACAATTATCCGTTTAACATGGATTTTGAAGCCCAGCAA...
7088,MEdarkturquoise,4,13106367,13107055,AT4G25730,TRM7B,CGTGCAGAACCAAATTAAAAGCACTAACACCATGCTGCTCCATCAC...


In [140]:
import pandas as pd
from Bio import SeqIO

def extract_non_variable_sequences_to_df(df, genome_path):
    # Load the genome fasta file
    genome_sequences = SeqIO.to_dict(SeqIO.parse(genome_path, "fasta"))
    
    # Initialize a list to store sequences
    sequences = []
    
    # Iterate through each row of the DataFrame
    for idx, row in df.iterrows():
        # Extract sequence details
        seq_name = row['seqnames']
        start = int(row['start'])
        end = int(row['end'])
        # gene_name = row['geneId']
        # gene = row['gene']
        
        # Extract the sequence from the genome fasta
        sequence = str(genome_sequences[seq_name].seq[start-1:end])  # -1 because Python is 0-based
        
        # Append the sequence to the list
        sequences.append(sequence)
    
    # Add the sequences as a new column to the DataFrame
    df['sequence'] = sequences
    
    return df

# Usage example:
# Assuming 'dataframe' is your existing DataFrame and 'path_to_genome.fasta' is the path to your fasta file
non_variable_peaks_in_RNA_seq2 = extract_non_variable_sequences_to_df(non_variable_peaks_in_RNA_seq, genome_path)


In [144]:
non_variable_peaks_in_RNA_seq2

,seqnames,start,end,width,strand,Conc,Conc_DM1.DM2d,Conc_Col.0_12DAS,Fold,p.value,FDR,annotation,geneChr,geneStart,geneEnd,geneLength,geneStrand,geneId,distanceToTSS,sequence
0,5,5236541,5237157,617,*,10.907429,9.407313,11.626810,-2.200833,6.460956e-278,1.128907e-273,Promoter (<=1kb),5,5236634,5236879,246,1,AT5G02495,0.0,AATTTCATAGTTAGTCATCTTCACCACATGTACAACAAAGATGAAT...
1,3,7104355,7104488,134,*,10.252167,8.418776,11.034068,-2.588420,1.021448e-256,8.923750e-253,Promoter (<=1kb),3,7103817,7104250,434,1,AT3G20362,538.0,GCAATATGTCTACGCTGGTTCAAATCCAGCTCGGCCCAATAATTAG...
2,5,17828736,17829549,814,*,11.143519,9.809886,11.824502,-1.997740,3.602299e-250,2.517683e-246,Promoter (1-2kb),5,17829618,17831336,1719,2,AT5G44260,1787.0,ACCGTAGTACCGTACGTACGTGGTCCTCACTCTCACATCTCAAAGA...
3,2,17504350,17505043,694,*,10.551486,8.694952,11.337129,-2.614031,5.171035e-250,3.285534e-246,Promoter (<=1kb),2,17504551,17504932,382,2,AT2G09320,0.0,AGTTTGGTCATGAGTTCGATGAAGTGATGGAGAGTTAAGAGTAGAT...
4,1,8598368,8599538,1171,*,11.514378,10.492546,12.106545,-1.601076,2.816934e-228,1.514449e-224,Promoter (<=1kb),1,8598624,8598990,367,1,AT1G05863,0.0,AGTCTAAAGCATAAGAAAAAAAAAGGTTAGTAAGAAAGAAAGAAAG...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24261,1,4036334,4037134,801,*,9.993201,10.086007,9.894011,0.164856,2.362045e-02,4.995784e-02,Promoter (1-2kb),1,4034479,4039065,4587,2,AT1G11950,1931.0,TGTTAAGAGCATTTCGAACAATCACTGGTTCACCTTTGGACCAATG...
24262,5,3658888,3658978,91,*,7.613293,7.426264,7.778834,-0.264245,2.362478e-02,4.996483e-02,Promoter (<=1kb),5,3656582,3659025,2444,2,AT5G11460,47.0,CGTATATATCTGTGCAAGTCTTGTGAGGTACTCTTCTCTCTCTCTC...
24263,5,13056298,13056771,474,*,9.006034,9.140994,8.857133,0.222461,2.362877e-02,4.996939e-02,Distal Intergenic,5,13061035,13062230,1196,1,AT5G34869,-4264.0,ATCTTAGGTTCATCTAAACCACGAAGACCAAACTTCTCCTCCACAT...
24264,1,29390905,29391031,127,*,7.891752,7.722579,8.043153,-0.246531,2.363869e-02,4.998885e-02,Promoter (<=1kb),1,29391630,29393334,1705,1,AT1G78110,-599.0,TATATAAGTATATAGTTTGGTGATATAGTTAGAAGGTAGTTAAATC...


In [145]:
def write_sequences_to_fasta(df, output_path):
    # Open the file in write mode
    with open(output_path, 'w') as fasta_file:
        # Iterate through the DataFrame
        for index, seq in df.iterrows():
            # Create the header
            header = f">{seq['geneId']}_{seq['seqnames']}_{seq['start']}_{seq['end']}"
            # Retrieve the sequence
            sequence = seq['sequence']  # Make sure 'sequence' is the correct column name where sequences are stored
            
            # Write the header and sequence to the file
            fasta_file.write(f"{header}\n{sequence}\n")

# Usage:
output_fasta_path = os.path.join(fasta_files_dir, f"non_variable_peaks_in_RNA_seq.fasta")
write_sequences_to_fasta(non_variable_peaks_in_RNA_seq2, output_fasta_path)



In [123]:
sequences_df

with open(os.path.join(fasta_files_dir, f"all_WGCNA_seq.fasta"), 'w') as fasta_file:
    # Iterate through the DataFrame
    n = 0
    for index, seq in sequences_df.iterrows():
        # Create the header
        header = f">{seq['gene']}_{seq['gene_name']}_{seq['seq_name']}_{seq['start']}_{seq['end']}"
        # Retrieve the sequence
        sequence = seq['peak_sequence']  # Make sure 'sequence' is the correct column name where sequences are stored
        # Write the header and sequence to the file
        fasta_file.write(f"{header}\n{sequence}\n")
        n += 1
    print(n, " sequences written to file")

7090  sequences written to file


In [93]:
# Assuming 'sequences_df' is your DataFrame

# Get the unique modules
unique_modules = sequences_df['module'].unique()

# Print the number of rows for each module
for module in unique_modules:
    n_rows = sequences_df[sequences_df['module'] == module].shape[0]
    print(f"Module {module} has {n_rows} rows")


Module MEcyan has 2859 rows
Module MEblue has 3219 rows
Module MEpurple has 332 rows
Module MEdarkred has 132 rows
Module MEgreenyellow has 117 rows
Module MEtan has 92 rows
Module MEdarkgreen has 129 rows
Module MEsalmon has 103 rows
Module MEmidnightblue has 74 rows
Module MEdarkturquoise has 33 rows


In [95]:
import os
import random

# Function to write sequences to a fasta file
def write_to_fasta(filename, sequences):
    with open(filename, 'w') as f:
        for seq in sequences:
            header = f">{seq['gene']}{seq['gene_name']}_{seq['seq_name']}_{seq['start']}_{seq['end']}"
            sequence = seq['peak_sequence']
            f.write(f"{header}\n{sequence}\n")

# Base directory for the fasta files
fasta_files_dir = './WGCNA_ATAC_integration/fasta_files_and_control/'
os.makedirs(fasta_files_dir, exist_ok=True)

# Get the unique modules
unique_modules = sequences_df['module'].unique()

# Dictionary to keep track of control sequences for each module
control_sequences = {module: [] for module in unique_modules}

# Iterate over each module and create fasta files
for module in unique_modules:
    module_df = sequences_df[sequences_df['module'] == module]
    n_sequences = module_df.shape[0]

    # Write the current module sequences to a fasta file
    fasta_filename = os.path.join(fasta_files_dir, f"{module}.fasta")
    write_to_fasta(fasta_filename, module_df.to_dict('records'))

    # Select random sequences from other modules
    other_modules_df = sequences_df[sequences_df['module'] != module]
    control_seqs = other_modules_df.sample(n_sequences).to_dict('records')
    control_sequences[module].extend(control_seqs)

    # Write the control sequences to a fasta file
    control_fasta_filename = os.path.join(fasta_files_dir, f"control_for_{module}.fasta")
    write_to_fasta(control_fasta_filename, control_seqs)

    print(f"Module {module}: {n_sequences} sequences")
    print(f"Control sequences written to {control_fasta_filename}")


Module MEcyan: 2859 sequences
Control sequences written to ./WGCNA_ATAC_integration/fasta_files_and_control/control_for_MEcyan.fasta
Module MEblue: 3219 sequences
Control sequences written to ./WGCNA_ATAC_integration/fasta_files_and_control/control_for_MEblue.fasta
Module MEpurple: 332 sequences
Control sequences written to ./WGCNA_ATAC_integration/fasta_files_and_control/control_for_MEpurple.fasta
Module MEdarkred: 132 sequences
Control sequences written to ./WGCNA_ATAC_integration/fasta_files_and_control/control_for_MEdarkred.fasta
Module MEgreenyellow: 117 sequences
Control sequences written to ./WGCNA_ATAC_integration/fasta_files_and_control/control_for_MEgreenyellow.fasta
Module MEtan: 92 sequences
Control sequences written to ./WGCNA_ATAC_integration/fasta_files_and_control/control_for_MEtan.fasta
Module MEdarkgreen: 129 sequences
Control sequences written to ./WGCNA_ATAC_integration/fasta_files_and_control/control_for_MEdarkgreen.fasta
Module MEsalmon: 103 sequences
Control sequ

### the bed file of peaks from Chipseeker

## homer2 compare enriched genes vs random background

In [160]:
#using homer provided motifs

def find_motifs_task(bed_file):
    # Assume bed_file path is like '.../WGCNA_integrated/bed_files/MEcyan.bed'
    module_name = os.path.splitext(os.path.basename(bed_file))[0]
    WGCNA_path = os.path.dirname(os.path.dirname(bed_file)) # Go up two levels to the WGCNA_integrated directory

    # Define unique output path for the module
    output_path = os.path.join(WGCNA_path, 'homer_using_homer_database_plants' ,module_name + '_motifs')
    
    # Create the output directory if it doesn't exist
    os.makedirs(output_path, exist_ok=True)
    
    #Program will find de novo and known motifs in regions in the genome
    cmd = [
        "findMotifsGenome.pl",
        bed_file,
         os.path.join('/media/HDD2/Genomes/Ath_Ensembl56', 'Arabidopsis_thaliana.TAIR10.dna.toplevel.fa'),
        output_path,
        "-p", "20",
        "-mset", "plants"
    ]
    result = subprocess.run(cmd, capture_output=True, text=True)
    
    # Log file named after the module
    log_file_path = os.path.join(output_path, f"{module_name}_HOMER.log")
    
    # Append both stderr and stdout to the log file
    with open(log_file_path, "a") as log_file:
        log_file.write(result.stdout)
        log_file.write(result.stderr)

def run_find_motifs(WGCNA_path):
    # Define the directory where all bed files are located
    bed_files_dir = os.path.join(WGCNA_path, 'bed_files')
    # Gather all bed files
    bed_files = [os.path.join(bed_files_dir, f) for f in os.listdir(bed_files_dir) if f.endswith('.bed')]

    # Use Pool to run the tasks in parallel
    with Pool(6) as p:  # Adjust number of processes if necessary
        p.map(find_motifs_task, bed_files)

# Usage:
run_find_motifs(WGCNA_path)


In [159]:
##using PlantTFDB motifs

def find_motifs_task(bed_file):
    # Assume bed_file path is like '.../WGCNA_integrated/bed_files/MEcyan.bed'
    module_name = os.path.splitext(os.path.basename(bed_file))[0]
    WGCNA_path = os.path.dirname(os.path.dirname(bed_file)) # Go up two levels to the WGCNA_integrated directory

    # Define unique output path for the module
    output_path = os.path.join(WGCNA_path, 'homer_plantTFDB', module_name + '_motifs')
    
    # Create the output directory if it doesn't exist
    os.makedirs(output_path, exist_ok=True)
    
    #Program will find de novo and known motifs in regions in the genome
    cmd = [
        "findMotifsGenome.pl",
        bed_file,
         os.path.join('/media/HDD2/Genomes/Ath_Ensembl56', 'Arabidopsis_thaliana.TAIR10.dna.toplevel.fa'),
        output_path,
        "-p", "20",
        # "-mset", "plants"
        '-mknown', '/media/HDD2/donghui/bulk_ATAC_DM1_DM2d/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme'
    ]
    result = subprocess.run(cmd, capture_output=True, text=True)
    
    # Log file named after the module
    log_file_path = os.path.join(output_path, f"{module_name}_HOMER.log")
    
    # Append both stderr and stdout to the log file
    with open(log_file_path, "a") as log_file:
        log_file.write(result.stdout)
        log_file.write(result.stderr)

def run_find_motifs(WGCNA_path):
    # Define the directory where all bed files are located
    bed_files_dir = os.path.join(WGCNA_path, 'bed_files')
    # Gather all bed files
    bed_files = [os.path.join(bed_files_dir, f) for f in os.listdir(bed_files_dir) if f.endswith('.bed')]

    # Use Pool to run the tasks in parallel
    with Pool(6) as p:  # Adjust number of processes if necessary
        p.map(find_motifs_task, bed_files)

# Usage:
run_find_motifs(WGCNA_path)


## MEME AME enrichment of motifs from peaks2

### MEME installed from conda seems to be broken, use docker version instead

In [80]:
! sudo chmod -R 777 /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/WGCNA_ATAC_integration/


[sudo] password for hu: 


docker takes sudo, can run visudo and add "yourusername ALL=(ALL) NOPASSWD: /usr/bin/docker"
then run the script docker_mem_ame.sh, which loops over all fasta files

In [146]:
sudo docker run -v /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/:/data memesuite/memesuite ame --oc /data/WGCNA_ATAC_integration/ame_WGCNA_vs_non_variable_peaks/ --control /data/WGCNA_ATAC_integration/fasta_files_and_control/non_variable_peaks_in_RNA_seq.fasta /data/WGCNA_ATAC_integration/fasta_files_and_control/all_WGCNA_seq.fasta /data/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme

SyntaxError: invalid syntax (1206664557.py, line 1)

/media/HDD2/donghui/bulk_ATAC_DM1_DM2d » mkdir WGCNA_ATAC_integration/ame_results_with_shuffle/                      hu@chaelab2
(work2) -------------------------------------------------------------------------------------------------------------------------
/media/HDD2/donghui/bulk_ATAC_DM1_DM2d » chmod -R 777 WGCNA_ATAC_integration/ame_results_with_shuffle/   
/media/HDD2/donghui/bulk_ATAC_DM1_DM2d » bash ./docker_mem_ame_with_shuffle.sh    

In [161]:
! fimo --oc ./WGCNA_ATAC_integration/fimo_results/MEblue\
/media/HDD2/donghui/bulk_ATAC_DM1_DM2d/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme\
./WGCNA_ATAC_integration/fasta_files_and_control/MEblue.fasta 

Using motif +AT1G01060 of width 10.
Using motif -AT1G01060 of width 10.
Finding best site passing the output threshold in each of the 3219 sequences.
Found a best site in 714 sequences.
Computing q-values.
# Computing q-values.
#   Estimating pi_0 from a uniformly sampled set of 10000 p-values.
#   Estimating pi_0.
# Minimal pi_zero = 0.886139
#   Estimated pi_0=0.900893
Using motif +AT1G01250 of width 15.
Using motif -AT1G01250 of width 15.
Finding best site passing the output threshold in each of the 3219 sequences.
Found a best site in 510 sequences.
Computing q-values.
# Computing q-values.
#   Estimating pi_0 from a uniformly sampled set of 10000 p-values.
#   Estimating pi_0.
# Minimal pi_zero = 1.00161
#   Estimated pi_0=1
Using motif +AT1G01260 of width 8.
Using motif -AT1G01260 of width 8.
Finding best site passing the output threshold in each of the 3219 sequences.
Found a best site in 360 sequences.
Computing q-values.
# Computing q-values.
#   Estimating pi_0 from a uniform

## extract peak fastas

In [164]:
# Load the RDS file into Python
peakAnno_tb_DM1_DM2d = pyreadr.read_r(os.path.join('./fimo/peakAnno_tb_DM1_DM2d.rds'))
peakAnno_tb_DM1_DM2d = peakAnno_tb_DM1_DM2d[None]


In [165]:
import pandas as pd
from Bio import SeqIO

def extract_sequences(df, genome_path):
    # Load the genome fasta file as a dictionary
    genome_sequences = SeqIO.to_dict(SeqIO.parse(genome_path, "fasta"))

    # Add a new column to the DataFrame for storing sequences
    df['sequence'] = ''

    # Iterate over rows in the DataFrame
    for index, row in df.iterrows():
        seq_name = row['seqnames']
        start = row['start']
        end = row['end']

        # Extract the sequence and store it in the DataFrame
        gene_seq = str(genome_sequences[str(seq_name)].seq[start-1:end])
        df.at[index, 'sequence'] = gene_seq

    return df

# Define the path to the FASTA file
genome_path = '/media/HDD2/Genomes/Ath_Ensembl56/Arabidopsis_thaliana.TAIR10.dna.toplevel.fa'

# Assuming 'peakAnno_tb_DM1_DM2d' is your DataFrame, call the function
# Update 'peakAnno_tb_DM1_DM2d' with your DataFrame variable name
sequences_df = extract_sequences(peakAnno_tb_DM1_DM2d, genome_path)
print(sequences_df)


      seqnames     start       end  width  \
0            4   1499634   1501162   1529   
1            5  17549546  17550412    867   
2            1  28051844  28052977   1134   
3            5   5236541   5237157    617   
4            4  18125528  18126153    626   
...        ...       ...       ...    ...   
31351        1  28403669  28404892   1224   
31352        5  21383686  21384233    548   
31353        5  13056298  13056771    474   
31354        1  29390905  29391031    127   
31355        3   6428336   6429592   1257   

                                                sequence  
0      CCTTCGTAGCGTCTCGCTTTGGACCTTGCAGGCATTTTCCGACACA...  
1      ACGATAAGACCCCATAGATAATGAACTGTCTAAAGACAAAACCTTT...  
2      TCTCTTGAGCGGTGTAGGAAGGTGAAGTAATAAATGAAGAGGATAC...  
3      AATTTCATAGTTAGTCATCTTCACCACATGTACAACAAAGATGAAT...  
4      AGAGCAGAAAGGAGTCAGTGATGAGTATATATAGATAAGAACATTA...  
...                                                  ...  
31351  GACTCTACACCCTTCGAGCATGAAAACAACATCAGCCAT

In [166]:
sequences_df

,seqnames,start,end,width,sequence
0,4,1499634,1501162,1529,CCTTCGTAGCGTCTCGCTTTGGACCTTGCAGGCATTTTCCGACACA...
1,5,17549546,17550412,867,ACGATAAGACCCCATAGATAATGAACTGTCTAAAGACAAAACCTTT...
2,1,28051844,28052977,1134,TCTCTTGAGCGGTGTAGGAAGGTGAAGTAATAAATGAAGAGGATAC...
3,5,5236541,5237157,617,AATTTCATAGTTAGTCATCTTCACCACATGTACAACAAAGATGAAT...
4,4,18125528,18126153,626,AGAGCAGAAAGGAGTCAGTGATGAGTATATATAGATAAGAACATTA...
...,...,...,...,...,...
31351,1,28403669,28404892,1224,GACTCTACACCCTTCGAGCATGAAAACAACATCAGCCATAGACGGA...
31352,5,21383686,21384233,548,AACTTTTACCTAGAGAACTTGCTTGAGAAGTTCTTATGACCGAAGA...
31353,5,13056298,13056771,474,ATCTTAGGTTCATCTAAACCACGAAGACCAAACTTCTCCTCCACAT...
31354,1,29390905,29391031,127,TATATAAGTATATAGTTTGGTGATATAGTTAGAAGGTAGTTAAATC...


In [167]:
def write_sequences_to_fasta(df, output_path):
    with open(output_path, 'w') as fasta_file:
        for index, row in df.iterrows():
            # Create a header using seqnames, start, and end
            header = f">{row['seqnames']}_{row['start']}_{row['end']}"
            sequence = row['sequence']

            # Write the header and sequence to the FASTA file
            fasta_file.write(f"{header}\n{sequence}\n")

# Usage example
output_fasta_path = "./fimo/sequences_for_all_peaks_DM1_DM2d.fasta"
write_sequences_to_fasta(sequences_df, output_fasta_path)
